In [2]:
import os
import glob
from datetime import datetime
from datetime import timezone
import polars as pl
import polars.selectors as cs
import plotly.express as px

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORy = os.environ.get("PICARRO_DATA_DIRECTORY")

In [3]:
# raw measurement data
df_raw = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet"))

# water corrected measurement data
df_dry = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "acropolis_dry.parquet"))

# slope and intercept data derived from calibrations
df_cal = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "slope_intercept_acropolis.parquet")) \
    .with_columns((pl.col("creation_timestamp").dt.timestamp("ms") / 3.6e6).alias("timestamp")) #3.6e6 ms are in 1h
    
# 10m preprocessed picarro data
df_p_10m = pl.read_parquet(os.path.join(DATA_DIRECTORY,"processed", "10m_cal_corr_picarro.parquet"))

In [20]:
def analyse_system(id, start_date, end_date):
    
    df_cal_temp = df_cal.filter(pl.col("creation_timestamp").is_between(start_date, end_date)) \
        .filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{id}") \
        .collect()

    # calculate calibration corrected dataset
    df_cal_corr = df_dry.filter(pl.col("gmp343_dry") > 0) \
        .filter(pl.col("creation_timestamp").is_between(start_date, end_date))  \
        .with_columns(pl.col("creation_timestamp").dt.date().alias("date")) \
        .collect() \
        .join(df_cal_temp, on = ["date","system_name"], how= "left") \
        .filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{id}") \
        .fill_null(strategy = "forward") \
        .fill_null(strategy = "backward") \
        .with_columns(((pl.col("gmp343_dry")) * pl.col("slope") + pl.col("intercept")) \
        .alias("CO2_corr")) \
        .sort("creation_timestamp") \
        .groupby_dynamic("creation_timestamp", every='10m')  \
        .agg([
            pl.all().exclude(["creation_timestamp","system_name"]).mean(),
            pl.col("system_name")
            ]) \
        .with_columns(pl.col("system_name").list.last()) \

        
    # calculate the difference to the picarro
    df_cal_corr = df_cal_corr.join(df_p_10m.select("creation_timestamp", "CO2_corr") \
        .rename({"CO2_corr": "temp"}), on="creation_timestamp", how= "left") \
        .with_columns((pl.col("CO2_corr") - pl.col("temp")).alias("diff")) \
        .drop("temp")
                
    # quantify the performance based on the calibration      
    df_cal_corr = df_cal_corr.groupby_dynamic("creation_timestamp", every='1d')  \
        .agg([
            pl.all().exclude(["creation_timestamp","system_name"]).mean()
            ]) \
    .with_columns(((pl.col("diff") < 1.3) & (pl.col("diff") > -1.3)).alias("calibration_accuracy"))

    # plot results from before
    fig = px.scatter(df_cal_corr, x="creation_timestamp", y="diff", title = "Difference System - Picarro", color="calibration_accuracy")
    fig.show()
    
    #--------------

    # extraxt for date and calibration accuracy information
    df_cal_acc = df_cal_corr.with_columns(pl.col("creation_timestamp").dt.date().alias("date")) \
        .select("date","calibration_accuracy")

    # join calibration accuracy into df_cal matched on date
    df_cal_temp = df_cal_temp.join(df_cal_acc, on = ["date"], how= "left")

    fig = px.scatter(df_cal_temp, x="creation_timestamp", y="slope", title = "Slope", color="calibration_accuracy")
    fig.show()

    fig = px.scatter(df_cal_temp, x="creation_timestamp", y="intercept", title = "Intercept", color="calibration_accuracy")
    fig.show()
    
    #--------------

    # selected calibration periods from raw auxialliary measurement data
    df_raw_temp = df_raw.select(cs.starts_with("cal_"), "creation_timestamp", "system_name")  \
        .filter(pl.col("cal_bottle_id") > 0) \
        .collect() \
        .filter(pl.col("creation_timestamp").is_between(start_date, end_date)) \
        .filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{id}") \
        .sort("creation_timestamp")  \
        .with_columns(pl.col("creation_timestamp").dt.date().alias("date"))  \


    # select the first timestamp of each calibration procedure
    df_calibration_start = df_raw_temp.select("creation_timestamp", "date") \
        .sort(pl.col("creation_timestamp")) \
        .groupby("date").first()

    # normalize all calibrations to start at same time
    df_raw_temp = df_raw_temp.join(df_calibration_start, on = ["date"], how= "left") \
        .with_columns((pl.col("creation_timestamp") - pl.col("creation_timestamp_right")).alias("time_since_calibration_start")) \
        .with_columns((datetime(1970, 1, 1, 0, 0, 0) + pl.col("time_since_calibration_start")).alias("normalised_time"))

    # join calibration accuracy into df_raw_temp matched on date
    df_raw_temp = df_raw_temp.join(df_cal_acc, on = ["date"], how= "left")  \

    fig = px.scatter(df_raw_temp, x="normalised_time", y="cal_sht45_humidity", title = "SHT45 Calibration Humidity", color="calibration_accuracy")
    fig.show()

    fig = px.scatter(df_raw_temp, x="normalised_time", y="cal_bme280_pressure", title = "BME280 Calibration Pressure", color="calibration_accuracy")
    fig.show()

    fig = px.scatter(df_raw_temp, x="normalised_time", y="cal_gmp343_temperature", title = "GMP343 Calibration Temperature", color="calibration_accuracy")
    fig.show()

    fig = px.scatter(df_raw_temp, x="normalised_time", y="cal_gmp343_filtered", title = "GMP343 Filtered Measurement Output", color="calibration_accuracy")
    fig.show()

## System 1

In [21]:
start_date = datetime(2024, 2, 7, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 13, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(1, start_date, end_date)

## System 3

In [11]:
start_date = datetime(2024, 1, 12, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 11, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(3, start_date, end_date)

## System 4

In [ ]:
start_date = datetime(2024, 2, 12, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 26, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(4, start_date, end_date)

## System 5

In [ ]:
start_date = datetime(2024, 2, 6, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 11, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(5, start_date, end_date)

## System 6

In [ ]:
start_date = datetime(2024, 1, 12, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 11, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(6,start_date, end_date)

## System 8

In [ ]:
start_date = datetime(2024, 2, 13, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 26, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(8, start_date, end_date)

## System 10

In [ ]:
start_date = datetime(2024, 1, 12, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 11, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(10, start_date, end_date)

## System 11

In [ ]:
start_date = datetime(2023, 12, 23, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 11, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(11, start_date, end_date)

# System 12

In [ ]:
id = 12

start_date = datetime(2023, 12, 23, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 11, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(12, start_date, end_date)

## System 13

In [ ]:
start_date = datetime(2024, 1, 12, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 5, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(13, start_date, end_date)

## System 16

In [ ]:
start_date = datetime(2023, 12, 23, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 5, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(16, start_date, end_date)

## System 18

In [ ]:
id = 18

start_date = datetime(2023, 12, 23, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 5, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(18, start_date, end_date)

## System 20

In [22]:
start_date = datetime(2023, 12, 23, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 2, 11, 23, 59, 59).replace(tzinfo=timezone.utc)

analyse_system(20, start_date, end_date)